In [1]:
# Data processing
import iris
import numpy as np
import xarray as xr
import dateutil.parser
import warnings
warnings.filterwarnings('ignore', module='iris')
from iris.experimental import stratify
from iris.analysis import trajectory
from pathlib import Path
from datetime import datetime, timedelta
## Scientific parameters
M_air = 28.97 # molar mass of dry air [g mol-1]
# Coefficients to convert mass mixing ratio to volume mixing ratio
coeff_c2h6 = (M_air/30.0690)*1e12
coeff_c3h8 = (M_air/44.0956)*1e12
coeff_nc4h10 = (M_air/58.1222)*1e12
coeff_ic4h10 = (M_air/58.1222)*1e12
coeff_nc5h12 = (M_air/72.1488)*1e12
coeff_ic5h12 = (M_air/72.1488)*1e12
coeff_meono2 = (M_air/77.0394)*1e12
coeff_sbuono2 = (M_air/119.1192)*1e12
coeff_n2peono2 = (M_air/133.1457)*1e12
coeff_n3peono2 = (M_air/133.1457)*1e12
coeff_ipeono2 = (M_air/133.1457)*1e12

#### Read data

In [2]:
path_to_atom = Path('../../../obs/ATom/nc/data')
# Select flight
fname_atom_flight = 'MER-WAS_DC8_20160729_R14.nc'
# Read ATom data
atom_dsinf = xr.open_dataset(path_to_atom / fname_atom_flight, decode_times=True)
atom_dsmms = xr.open_dataset(path_to_atom / fname_atom_flight, group='MMS', decode_cf=True)

In [3]:
# Choose UKCA run
ukca_run_name = 'az166a'
ukca_run_yrmn = 'pm2016jul'
path_to_ukca = Path('../data') / ukca_run_name
fname_ukca_run = f'{ukca_run_name}.{ukca_run_yrmn}.pp'
# Read UKCA data
cb_c2h6 = iris.load_cube(str(path_to_ukca / fname_ukca_run), 'mass_fraction_of_ethane_in_air')*coeff_c2h6
cb_c3h8 = iris.load_cube(str(path_to_ukca / fname_ukca_run), 'mass_fraction_of_propane_in_air')*coeff_c3h8
cb_nc4h10 = iris.load_cube(str(path_to_ukca / fname_ukca_run), iris.AttributeConstraint(STASH='m01s34i173'))*coeff_nc4h10
cb_ic4h10 = iris.load_cube(str(path_to_ukca / fname_ukca_run), iris.AttributeConstraint(STASH='m01s34i174'))*coeff_ic4h10
cb_nc5h12 = iris.load_cube(str(path_to_ukca / fname_ukca_run), iris.AttributeConstraint(STASH='m01s34i175'))*coeff_nc5h12
cb_ic5h12 = iris.load_cube(str(path_to_ukca / fname_ukca_run), iris.AttributeConstraint(STASH='m01s34i176'))*coeff_ic5h12
cb_meono2 = iris.load_cube(str(path_to_ukca / fname_ukca_run), 'mass_fraction_of_methyl_nitrate_in_air')*coeff_meono2
cb_sbuono2 = iris.load_cube(str(path_to_ukca / fname_ukca_run), iris.AttributeConstraint(STASH='m01s34i188'))*coeff_sbuono2
cb_n2peono2 = iris.load_cube(str(path_to_ukca / fname_ukca_run), iris.AttributeConstraint(STASH='m01s34i189'))*coeff_n2peono2
cb_n3peono2 = iris.load_cube(str(path_to_ukca / fname_ukca_run), iris.AttributeConstraint(STASH='m01s34i190'))*coeff_n3peono2
cb_ipeono2 = iris.load_cube(str(path_to_ukca / fname_ukca_run), iris.AttributeConstraint(STASH='m01s34i191'))*coeff_ipeono2

In [18]:
# Extract datetimes from ATom and UKCA data
atom_datetime = atom_dsinf.time.values.astype('<M8[us]').astype(datetime)
atom_date_strt = atom_datetime[0]
atom_date_stop = atom_datetime[-1]
ukca_days_since = dateutil.parser.parse(str(cb_nc4h10.coord('time').units)[12:])
_ukca_datetime = []
for i, ii in enumerate(list(cb_nc4h10.coord('time').points)):
    _ukca_datetime.append(ukca_days_since + timedelta(hours=float(f'{ii:1.3f}'))) # UKCA's 'hours since 1970-01-01' format is 1.3f
ukca_datetime = np.array(_ukca_datetime, dtype=datetime)
# Create a common timestamp/flight merge identifier for trajectories
compared_common_id = f'{atom_date_strt.strftime("%y%m%d")}_{atom_date_stop.strftime("%y%m%d")}_{ukca_datetime[0].strftime("%y%m")}_singl'
print(compared_common_id)

160729_160729_1607_singl


#### Process data
##### 1. Relevel model data from level height to absolute height 

In [ ]:
# Relevel model data from level height to target levels
tgt_levels = np.arange(0, 14000, 100) # [meters]
cbr_c2h6 = stratify.relevel(cb_c2h6, cb_c2h6.coord('altitude'), tgt_levels, axis='level_height')
cbr_c3h8 = stratify.relevel(cb_c3h8, cb_c3h8.coord('altitude'), tgt_levels, axis='level_height')
cbr_nc4h10 = stratify.relevel(cb_nc4h10, cb_nc4h10.coord('altitude'), tgt_levels, axis='level_height')
cbr_ic4h10 = stratify.relevel(cb_ic4h10, cb_ic4h10.coord('altitude'), tgt_levels, axis='level_height')
cbr_nc5h12 = stratify.relevel(cb_nc5h12, cb_nc5h12.coord('altitude'), tgt_levels, axis='level_height')
cbr_ic5h12 = stratify.relevel(cb_ic5h12, cb_ic5h12.coord('altitude'), tgt_levels, axis='level_height')
cbr_meono2 = stratify.relevel(cb_meono2, cb_meono2.coord('altitude'), tgt_levels, axis='level_height')
cbr_sbuono2 = stratify.relevel(cb_sbuono2, cb_sbuono2.coord('altitude'), tgt_levels, axis='level_height')
cbr_n2peono2 = stratify.relevel(cb_n2peono2, cb_n2peono2.coord('altitude'), tgt_levels, axis='level_height')
cbr_n3peono2 = stratify.relevel(cb_n3peono2, cb_n3peono2.coord('altitude'), tgt_levels, axis='level_height')
cbr_ipeono2 = stratify.relevel(cb_ipeono2, cb_ipeono2.coord('altitude'), tgt_levels, axis='level_height')

##### 2. Calculate trajectories

In [21]:
# Reconstruct a flight trajectory from ATom geospacial and datetime data
if compared_common_id == '160803_160803_1608_singl':
    # Because ATom flight on 20160803 has one lat, lon, alt value equal to 'nan', have to interpolate between adjacent cells
    # to replace 'nan' with an interpolated value.
    sample_lats = atom_dsmms.G_LAT.interpolate_na(dim='time', method='linear').data
    sample_lons = atom_dsmms.G_LONG.interpolate_na(dim='time', method='linear').data+360
    sample_alts = atom_dsmms.G_ALT.interpolate_na(dim='time', method='linear').data
else:
    sample_lats = atom_dsmms.G_LAT.data 
    sample_lons = atom_dsmms.G_LONG.data+360
    sample_alts = atom_dsmms.G_ALT.data
sample_lats_lons_alts = [('latitude', sample_lats), ('longitude', sample_lons), ('altitude', sample_alts)]

In [27]:
# Interpolate releveled model data to flight trajectory
traj_c2h6 = trajectory.interpolate(cbr_c2h6, sample_lats_lons_alts)
traj_c3h8 = trajectory.interpolate(cbr_c3h8, sample_lats_lons_alts)
traj_nc4h10 = trajectory.interpolate(cbr_nc4h10, sample_lats_lons_alts)
traj_ic4h10 = trajectory.interpolate(cbr_ic4h10, sample_lats_lons_alts)
traj_nc5h12 = trajectory.interpolate(cbr_nc5h12, sample_lats_lons_alts)
traj_ic5h12 = trajectory.interpolate(cbr_ic5h12, sample_lats_lons_alts)
traj_meono2 = trajectory.interpolate(cbr_meono2, sample_lats_lons_alts)
traj_sbuono2 = trajectory.interpolate(cbr_sbuono2, sample_lats_lons_alts)
traj_n2peono2 = trajectory.interpolate(cbr_n2peono2, sample_lats_lons_alts)
traj_n3peono2 = trajectory.interpolate(cbr_n3peono2, sample_lats_lons_alts)
traj_ipeono2 = trajectory.interpolate(cbr_ipeono2, sample_lats_lons_alts)

##### 3. Extract cross sections

In [28]:
# Extract a cross section from model data along flight track
sample_lats_lons = [('latitude', sample_lats), ('longitude', sample_lons)]
crsc_c2h6 = trajectory.interpolate(cbr_c2h6, sample_lats_lons)
crsc_c3h8 = trajectory.interpolate(cbr_c3h8, sample_lats_lons)
crsc_nc4h10 = trajectory.interpolate(cbr_nc4h10, sample_lats_lons)
crsc_ic4h10 = trajectory.interpolate(cbr_ic4h10, sample_lats_lons)
crsc_nc5h12 = trajectory.interpolate(cbr_nc5h12, sample_lats_lons)
crsc_ic5h12 = trajectory.interpolate(cbr_ic5h12, sample_lats_lons)
crsc_meono2 = trajectory.interpolate(cbr_meono2, sample_lats_lons)
crsc_sbuono2 = trajectory.interpolate(cbr_sbuono2, sample_lats_lons)
crsc_n2peono2 = trajectory.interpolate(cbr_n2peono2, sample_lats_lons)
crsc_n3peono2 = trajectory.interpolate(cbr_n3peono2, sample_lats_lons)
crsc_ipeono2 = trajectory.interpolate(cbr_ipeono2, sample_lats_lons)

#### Save processed data

In [29]:
path_to_prcd = Path('../processed') / ukca_run_name

In [31]:
# Save trajectories to .nc
iris.save(traj_c2h6, str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_traj_c2h6.nc')
iris.save(traj_c3h8, str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_traj_c3h8.nc')
iris.save(traj_nc4h10, str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_traj_nc4h10.nc')
iris.save(traj_ic4h10, str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_traj_ic4h10.nc')
iris.save(traj_nc5h12, str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_traj_nc5h12.nc')
iris.save(traj_ic5h12, str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_traj_ic5h12.nc')
iris.save(traj_meono2, str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_traj_meono2.nc')
iris.save(traj_sbuono2, str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_traj_sbuono2.nc')
iris.save(traj_n2peono2, str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_traj_n2peono2.nc')
iris.save(traj_n3peono2, str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_traj_n3peono2.nc')
iris.save(traj_ipeono2, str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_traj_ipeono2.nc')

In [32]:
# Save cross sections to .nc
iris.save(crsc_c2h6, str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_crsc_c2h6.nc')
iris.save(crsc_c3h8, str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_crsc_c3h8.nc')
iris.save(crsc_nc4h10, str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_crsc_nc4h10.nc')
iris.save(crsc_ic4h10, str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_crsc_ic4h10.nc')
iris.save(crsc_nc5h12, str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_crsc_nc5h12.nc')
iris.save(crsc_ic5h12, str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_crsc_ic5h12.nc')
iris.save(crsc_meono2, str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_crsc_meono2.nc')
iris.save(crsc_sbuono2, str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_crsc_sbuono2.nc')
iris.save(crsc_n2peono2, str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_crsc_n2peono2.nc')
iris.save(crsc_n3peono2, str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_crsc_n3peono2.nc')
iris.save(crsc_ipeono2, str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_crsc_ipeono2.nc')